In [29]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from sklearn.utils import shuffle

In [30]:
df0 = pd.read_csv("E:/CNSI_project/demo_and_data_5.0/Processed_data/MIMIC/aim2_dataAll_filtered_NameShorted.csv")
df0

,subject_id,hadm_id,chartdate,B_Albumin,B_AlkalinePhosphatase,B_Calcium,B_Chloride,B_LactateDehydrogenase,B_Phosphate,B_UreaNitrogen,...,age,resprate,acuity,Chief Complaint,icd_title,History of Present Illness,Past Medical History,Allergies,Family History,hospital_expire_flag
0,10001217,24597018,2157/11/19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,55,16.0,3.0,"LEFT HAND PAINS, Left hand and face numbness, ...",Cerebral edema; Compression of brain; Family h...,Mrs. ___ is a ___ y/o F from ___ with history ...,Multiple sclerosis,Penicillins,"Mother with pancreatic cancer, brother-lung ca...",0
1,10003019,21616816,2174/9/18,2.6,NaN,10.4,102.0,NaN,2.8,32.0,...,72,16.0,2.0,"FEVER/MS CHANGES, confusion, rehab placement","Acute kidney failure, unspecified; Asthma, uns...","___ male w/ PMH of sarcoidosis, sleep apnea, n...","1. Sarcoidosis, dx skin bx:intestinal & pulmon...",Oxycodone / Ragweed,"Mother:___, cardiac disease. Father:diverticul...",0
2,10005808,26543907,2119/1/2,NaN,48.0,9.3,105.0,NaN,2.5,18.0,...,53,NaN,NaN,limb weakness and tingling,Acute infective polyneuritis,Mr. ___ is a ___ year old right handed gentlem...,Hiatal hernia in ___ Occasional heartburn,No Known Allergies / Adverse Drug Reactions,He has no family history of autoimmune disease...,0
3,10006269,27357430,2124/6/26,4.9,80.0,NaN,95.0,NaN,NaN,15.0,...,51,19.0,3.0,"Fever, Headache",Duodenitis without bleeding; Essential (primar...,NaN,NaN,No Known Allergies / Adverse Drug Reactions,NaN,0
4,10014078,25809882,2166/8/22,NaN,NaN,8.1,117.0,NaN,2.8,7.0,...,60,NaN,NaN,seizure,Benign neoplasm of cerebral meninges; Diabetes...,The pt is a ___ year-old woman who presents as...,hx of atypical meningioma - grade 2 - resected...,No Known Allergies / Adverse Drug Reactions,noncontributory,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5790,19994772,29219051,2181/4/18,NaN,NaN,NaN,103.0,150.0,NaN,9.0,...,55,20.0,2.0,Aletered mental status and fever,Abnormal involuntary movements; Candidiasis of...,___ is a ___ woman with,PAST MEDICAL HISTORY:Recurrent glioblastoma in...,Keflex,Non-contributory.,0
5791,19994772,29606061,2180/12/18,NaN,NaN,9.7,103.0,NaN,3.3,16.0,...,54,NaN,NaN,Recurrent glioblastoma.,"Cerebral aneurysm, nonruptured; Compression of...",Ms. ___ is a ___ y/o F s/p resection of gliobl...,Hyperlipidemia,___,non-contributory,0
5792,19995080,23057109,2147/7/13,2.3,961.0,7.5,105.0,NaN,2.3,17.0,...,59,NaN,NaN,Confusion- Altered Mental Status- transfer fro...,Calculus of gallbladder and bile duct without ...,___ M with AIDS and med non-compliance (CD4 20...,"- AIDS off ART prescribed for Truvada T qd, Ka...",Penicillins,Father died of AD at age ___. Mother died of D...,0
5793,19999784,24935234,2119/7/7,4.3,NaN,NaN,NaN,102.0,NaN,NaN,...,57,16.0,2.0,"Abnormal MRI, weakness",Abnormal cytological findings in cerebrospinal...,Mr. ___ is a ___ without significant medical h...,PAST ONCOLOGIC HISTORY:___ is a ___ right-hand...,No Known Allergies / Adverse Drug Reactions,Father had prostate cancer. Denies otherwise ...,0


In [31]:
df0.hospital_expire_flag.value_counts()

0    5500
1     295
Name: hospital_expire_flag, dtype: int64

In [32]:
########### dataset 1 ########
df1 = df0.copy()
df1 = df1[["Chief Complaint","hospital_expire_flag"]]
df2 = df1.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# df2 = df2[df2['hospital_expire_flag'].notnull()]
# df2 = df2.dropna(thresh=2,axis=0)   # without even one Nan value
df2['hospital_expire_flag'] = df2['hospital_expire_flag'].astype(np.int16)
df2.rename(columns={'Chief Complaint':'English'},inplace=True)
df2

,English,hospital_expire_flag
0,"left hand pains, left hand and face numbness, ...",0
1,"fever/ms changes, confusion, rehab placement",0
2,limb weakness and tingling,0
3,"fever, headache",0
4,seizure,0
...,...,...
5790,aletered mental status and fever,0
5791,recurrent glioblastoma.,0
5792,confusion- altered mental status- transfer fro...,0
5793,"abnormal mri, weakness",0


# BioClinBert

In [33]:
pretrained_model = 'G:/BertModels/Bio_ClinicalBERT'
# pretrained_model = 'emilyalsentzer/Bio_ClinicalBERT'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [34]:
def text2vec(df_text, col, token_length):
    #col = "Chief Complaint"
    sentences_maxlength = 1398
    sentences = df_text[col].fillna("").values.tolist()
    encoding = tokenizer(sentences, 
                         add_special_tokens=False, 
                         return_tensors='tf', 
                         padding='max_length', 
                         truncation=True, 
                         max_length=sentences_maxlength)

    input_ids = encoding['input_ids']
    input_ids1 = input_ids.numpy()
    
#     ICD:34, ChiefComplaint:20
    input_ids1_cumF95 = input_ids1[:,:token_length]
    
    return input_ids1_cumF95

In [35]:
X_train = text2vec(df2, 'English', 379)
BioClinBert = X_train.copy()
BioClinBert

array([[ 1286,  1289, 24782, ...,     0,     0,     0],
       [10880,   120,   182, ...,     0,     0,     0],
       [16864, 11477,  1105, ...,     0,     0,     0],
       ...,
       [ 6406,   118,  8599, ...,     0,     0,     0],
       [22832,   182,  2047, ...,     0,     0,     0],
       [  181,  3420, 11477, ...,     0,     0,     0]])

# BioBert

In [36]:
pretrained_model = 'G:/BertModels/biobert-v1.1'
# pretrained_model = 'dmis-lab/biobert-v1.1'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [37]:
X_train = text2vec(df2, 'English', 379)
BioBert = X_train.copy()
BioBert

array([[ 1286,  1289, 24782, ...,     0,     0,     0],
       [10880,   120,   182, ...,     0,     0,     0],
       [16864, 11477,  1105, ...,     0,     0,     0],
       ...,
       [ 6406,   118,  8599, ...,     0,     0,     0],
       [22832,   182,  2047, ...,     0,     0,     0],
       [  181,  3420, 11477, ...,     0,     0,     0]])

In [38]:
(BioClinBert==BioBert).all()

True